In [11]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim import models
from collections import Counter
import pandas as pd
import numpy as np
import re
import os

In [12]:
print("Initializing drivers...")
ws_driver = CkipWordSegmenter(model="albert-base")
pos_driver = CkipPosTagger(model="albert-base")
print("Initializing drivers... done")

Initializing drivers...


KeyboardInterrupt: 

In [ ]:
def process_article(article):
    if not article:
        return ""
    # 斷詞
    ws = ws_driver([article])
    # 詞性標注
    pos = pos_driver(ws)

    # 只留下名詞和動詞，並去掉特定詞性、一個字詞
    short_sentence = []
    stop_pos = set([]) #不使用停用詞
    for sentence_ws, sentence_pos in zip(ws, pos):
        for word_ws, word_pos in zip(sentence_ws, sentence_pos):
            is_N_or_V = word_pos.startswith("V") or word_pos.startswith("N")
            is_not_stop_pos = word_pos not in stop_pos
            is_not_one_charactor = len(word_ws) > 1
            if is_N_or_V and is_not_stop_pos and is_not_one_charactor:
                short_sentence.append(word_ws)
    # 回傳斷詞後的結果
    return " ".join(short_sentence)

In [ ]:
count = 0
directory = './data/csv/'

for file in os.listdir(directory):
    dataset = pd.read_csv('./data/csv/{}'.format(file))
    dataset["content"].fillna("", inplace=True)
    content = dataset["content"].apply(lambda x: re.sub("[^\w\s\(\)\*\+\?\.\|]", "", str(x)))
    seg_result = content.apply(process_article)

    seg_result.to_csv('./data/seg_result/{}'.format(file), index=False)

    print(f"{count}:{file}已輸出")
    count = count + 1


In [ ]:
word_list = []
directory = './data/csv/'

for file in os.listdir(directory):
    article = []
    dataset = pd.read_csv('./data/csv/{}'.format(file))
    dataset["content"].fillna("", inplace=True)
    content = dataset["content"].apply(lambda x: re.sub("[^\w\s\(\)\*\+\?\.\|]", "", str(x)))
    seg_result = content.apply(process_article)
    
    for word in seg_result:
        if word == "": continue
        for ws in word.split(' '):
            if ws != "":
                article.append(ws)
    word_list.append(article)

print(len(word_list))

In [ ]:
model = Word2Vec(word_list, window=10, min_count=1, workers=6)
vocab = model.wv.index_to_key
vectors = model.wv[vocab]
vectors

In [ ]:
model.save('model/word2vec_gensim.model')